In [1]:
# Importar bibliotecas
import csv
import sqlite3

In [2]:
# Criar conexão com o banco
connection = sqlite3.connect("api_ifood.db",isolation_level=None) # isolation_level é o controle de concorrencia


# Criar objeto cursor, para usar o método cursor()  
cursor = connection.cursor()

In [3]:
# Criar tabela
cursor.execute("""CREATE TABLE IF NOT EXISTS base_1(
                    availableForScheduling TEXT
                    , avatar TEXT
                    , category TEXT
                    , delivery_fee NUMERIC
                    , delivery_time NUMERIC
                    , distance NUMERIC
                    , ibge NUMERIC
                    , minimumOrderValue NUMERIC
                    , name TEXT
                    , paymentCodes TEXT
                    , price_range TEXT
                    , rating NUMERIC
                    , tags TEXT
                    , url TEXT
               );
               """)


In [4]:
# Testar erros
try:
  
    # Importar o csv e extrair os dados
    with open("ifood-restaurants-february-2021.csv", "r", encoding='utf-8') as arq:
        arquivo = csv.DictReader(arq)
        dados_banco = [(
                        i['availableForScheduling']
                        , i['avatar']
                        , i['category'] 
                        , i['delivery_fee']
                        , i['delivery_time'] 
                        , i['distance']
                        , i['ibge']
                        , i['minimumOrderValue']
                        , i['name']
                        , i['paymentCodes']
                        , i['price_range']
                        , i['rating']
                        , i['tags']
                        , i['url']
                        ) for i in arquivo]
  
    # Inserir dados na tabela
    cursor.executemany("""
                            INSERT INTO base_1 (availableForScheduling
                                                , avatar
                                                , category
                                                , delivery_fee
                                                , delivery_time
                                                , distance
                                                , ibge
                                                , minimumOrderValue
                                                , name
                                                , paymentCodes
                                                , price_range
                                                , rating
                                                , tags
                                                , url)
                       
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                       ;""", dados_banco)
  
except sqlite3.Error as error:
    print("Error occured - ", error)
  


In [26]:
# Ver tebela
cursor.execute("SELECT * FROM base_1;")
  
# Ver resultado
resultado = cursor.fetchall()
print(resultado)

[]


In [8]:
# Tipos das colunas
cursor.execute("""
                    SELECT typeof(delivery_fee)
                    FROM base_1
               ;""")

# Ver resultado
resultado = cursor.fetchall()
print(resultado)

[('real',), ('real',), ('text',), ('real',), ('real',), ('text',)]


In [9]:
# Quantidade de NULL por coluna
cursor.execute("""
                    SELECT 
                        SUM(CASE WHEN delivery_fee IS NULL THEN 1 ELSE 0 END) as countar_nulls
                        , SUM(CASE WHEN delivery_fee IS NULL THEN 0 ELSE 1 END) as countar__n_nulls
                    FROM base_1
               ;""")

# Ver resultado
resultado = cursor.fetchall()
print(resultado)

[(0, 6)]


In [17]:
# Quantos valores únicos nas colunas categóricas
cursor.execute("""
                    SELECT 
                        COUNT(DISTINCT category)
                    FROM base_1
               ;""")

# Ver resultado
resultado = cursor.fetchall()
print(resultado)

   0
0  3


In [ ]:
# 10 perguntas do dataset, respondidas com query.

In [39]:
# Commit o trabalho
connection.commit()

# Fechar a cursor
cursor.close()

# Fechar a conexão
connection.close()
print('Conexão do SQLite encerrada.')

Conexão do SQLite encerrada.
